## Import Libraries
In this section, we import all necessary Python libraries required for our project. These libraries facilitate neural network construction (PyTorch), data handling (NumPy), and operating system interactions (os). We also import utilities for dataset management and splitting.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import os
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

## Load and Preprocess Data
This cell defines the `load_data` function to navigate through dataset directories, load binary sensor data files, and preprocess them. The preprocessing involves reshaping and scaling the data:
- Accelerometer data is scaled by 2048 to convert to 'g' units.
- Gyroscopic data is scaled by 16.4 to convert to degrees per second.
We store the scaled data and associated labels in NumPy arrays.

In [ ]:
def load_data(directory):
    data = []
    labels = []
    categories = ['axel', 'flip', 'loop', 'lutz', 'salchow', 'toe']
    for idx, category in enumerate(categories):
        path = os.path.join(directory, category)
        for file in os.listdir(path):
            file_path = os.path.join(path, file)
            with open(file_path, 'rb') as f:
                raw_data = np.fromfile(f, dtype=np.int16).reshape(150, 30)
                raw_data = raw_data.astype(np.float64)  # Convert to float64 before scaling
                # Apply scaling for accelerometer and gyroscopic data
                for i in range(5):
                    # Accelerometer indices: 0, 1, 2, 6, 7, 8, ..., 24, 25, 26
                    # Gyroscopic indices: 3, 4, 5, 9, 10, 11, ..., 27, 28, 29
                    acc_indices = [i*6, i*6+1, i*6+2]
                    gyro_indices = [i*6+3, i*6+4, i*6+5]
                    raw_data[:, acc_indices] /= 2048.0  # Scale accelerometer data
                    raw_data[:, gyro_indices] /= 16.4   # Scale gyroscopic data
                data.append(raw_data)
                labels.append(idx)
    return np.array(data), np.array(labels)

data, labels = load_data('../data/labeled_data')

## Define the LSTM Model
Below, we define our `JumpClassifier` class, which is a PyTorch neural network model consisting of:
- An LSTM layer to process sequences of IMU sensor data.
- A fully connected layer to classify the sequences into one of the six jump categories.

In [ ]:
class JumpClassifier(nn.Module):
    def __init__(self):
        super(JumpClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=30, hidden_size=50, num_layers=4, batch_first=True)
        self.fc = nn.Linear(50, 6)  # 6 categories

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # Get last time step
        x = self.fc(x)
        return x

model = JumpClassifier()

## Prepare Data for Training
This cell converts our loaded data into PyTorch tensors, splits it into training and validation sets, and prepares DataLoader objects for efficient data handling during model training.

In [ ]:
tensor_data = torch.Tensor(data)
tensor_labels = torch.LongTensor(labels)

X_train, X_val, y_train, y_val = train_test_split(tensor_data, tensor_labels, test_size=0.3, random_state=42)
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

## Training Loop
Here, we define and execute the training loop for our model. The process involves:
- Iterating over batches of training data.
- Computing the loss and updating the model parameters using backpropagation.
- Logging the loss to monitor the training progress.
We use a CrossEntropyLoss for classification and the Adam optimizer.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(200):  # Number of epochs
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

## Evaluate the Model
After training, we evaluate the model's accuracy on the validation set. This step is crucial to understand the effectiveness of our model and identify any issues such as overfitting.

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')